#### This notebook demonstrates the use of the Reject Option Classification (ROC) post-processing algorithm for bias mitigation.
- The debiasing function used is implemented in the `RejectOptionClassification` class.
- Divide the dataset into training, validation, and testing partitions.
- Train classifier on original training data.
- Estimate the optimal classification threshold, that maximizes balanced accuracy without fairness constraints.
- Estimate the optimal classification threshold, and the critical region boundary (ROC margin) using a validation set for the desired constraint on fairness. The best parameters are those that maximize the classification threshold while satisfying the fairness constraints.
- The constraints can be used on the following fairness measures:
    * Statistical parity difference on the predictions of the classifier
    * Average odds difference for the classifier
    * Equal opportunity difference for the classifier
- Determine the prediction scores for testing data. Using the estimated optimal classification threshold, compute accuracy and fairness metrics.
- Using the determined optimal classification threshold and the ROC margin, adjust the predictions. Report accuracy and fairness metric on the new predictions.

In [6]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
from tqdm import tqdm
from warnings import warn

from aif360.datasets import BinaryLabelDataset
from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.algorithms.postprocessing.reject_option_classification\
        import RejectOptionClassification
from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider
import pandas as pd
import pickle
from sklearn.linear_model import Lasso

## Huangrui's Dataset 

In [7]:
class GermanDataset(StandardDataset):
    """German credit Dataset.

    See :file:`aif360/data/raw/german/README.md`.
    """

    def __init__(self, path, label_name='Y', favorable_classes=[1],
                 protected_attribute_names=['sex', 'age'],
                 privileged_classes=[[1],[1]],
                 instance_weights_name=None,
                 categorical_features=[],
                 features_to_keep=[], features_to_drop=[],
                 na_values=[], custom_preprocessing=None,
                 metadata=None):
        
        df = pd.read_csv(path)
        

        super(GermanDataset, self).__init__(df=df, label_name=label_name,
            favorable_classes=favorable_classes,
            protected_attribute_names=protected_attribute_names,
            privileged_classes=privileged_classes,
            instance_weights_name=instance_weights_name,
            categorical_features=categorical_features,
            features_to_keep=features_to_keep,
            features_to_drop=features_to_drop, na_values=na_values,
            custom_preprocessing=custom_preprocessing, metadata=metadata)

#### Load dataset and specify options

In [8]:
## import dataset
dataset_used = "german" # "german", "german", "compas"
protected_attribute_used = 1 # 1, 2


#     dataset_orig = GermanDataset()
if protected_attribute_used == 1:
    privileged_groups = [{'sex': 1}]
    unprivileged_groups = [{'sex': 0}]
else:
    privileged_groups = [{'age': 1}]
    unprivileged_groups = [{'age': 0}]
        
# Metric used (should be one of allowed_metrics)
metric_name = "Equal opportunity difference"

# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05
        
        
#random seed for calibrated equal odds prediction
random_seed = 12345679
np.random.seed(random_seed)

# Verify metric name
allowed_metrics = ["Statistical parity difference",
                   "Average odds difference",
                   "Equal opportunity difference"]
if metric_name not in allowed_metrics:
    raise ValueError("Metric name should be one of allowed metrics")

#### Split into train, test and validation

In [9]:
experiments_info = {}
budget = 0.1
for K in range(1, 6):
    if protected_attribute_used == 1:
        dataset_orig_train= GermanDataset(path="./Huangrui/german/german_train{}.csv".format(K),protected_attribute_names=['sex'],
                    privileged_classes=[[1]])
        dataset_orig_test= GermanDataset(path="./Huangrui/german/german_test{}.csv".format(K),protected_attribute_names=['sex'],
                    privileged_classes=[[1]])
    else:
        dataset_orig_train= GermanDataset(path="./Huangrui/german/german_train{}.csv".format(K),protected_attribute_names=['age'],
                    privileged_classes=[[1]])
        dataset_orig_test= GermanDataset(path="./Huangrui/german/german_test{}.csv".format(K),protected_attribute_names=['age'],
                    privileged_classes=[[1]])
    #only use the budget% of the training data
    dataset_orig_train,_ = dataset_orig_train.split([budget], shuffle=False)

    # Lasso linear classifier and predictions
    scale_orig = StandardScaler()
    X_train = scale_orig.fit_transform(dataset_orig_train.features)
    y_train = dataset_orig_train.labels.ravel()
    if protected_attribute_used == 1:
        lmod = pickle.load(open('experiments/german'+str(K)+'_sex_bmodel.pkl','rb'))["clf"]
    else:
        lmod = pickle.load(open('experiments/german'+str(K)+'_age_bmodel.pkl','rb'))["clf"]
    y_train_pred = lmod.predict(X_train)

    dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
    dataset_orig_train_pred.labels = y_train_pred.reshape(-1,1)
    sigmoid = lambda x: 1 / (1 + np.exp(0.5-x))
    dataset_orig_train_pred.scores = sigmoid(y_train_pred).reshape(-1,1)

    dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
    X_test = scale_orig.transform(dataset_orig_test_pred.features)
    y_test = dataset_orig_test_pred.labels
    dataset_orig_test_pred.scores = sigmoid(lmod.predict(X_test)).reshape(-1,1)

    #### Best threshold for classification only (no fairness)
    num_thresh = 100
    ba_arr = np.zeros(num_thresh)
    class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
    for idx, class_thresh in enumerate(class_thresh_arr):
        
        fav_inds = dataset_orig_train_pred.scores > class_thresh
        dataset_orig_train_pred.labels[fav_inds] = dataset_orig_train_pred.favorable_label
        dataset_orig_train_pred.labels[~fav_inds] = dataset_orig_train_pred.unfavorable_label
        
        classified_metric_orig_train = ClassificationMetric(dataset_orig_train,
                                                dataset_orig_train_pred, 
                                                unprivileged_groups=unprivileged_groups,
                                                privileged_groups=privileged_groups)
        
        ba_arr[idx] = 0.5*(classified_metric_orig_train.true_positive_rate()\
                        +classified_metric_orig_train.true_negative_rate())

    best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
    best_class_thresh = class_thresh_arr[best_ind]

    # print("Best balanced accuracy (no fairness constraints) = %.4f" % np.max(ba_arr))
    # print("Optimal classification threshold (no fairness constraints) = %.4f" % best_class_thresh)
    #### Estimate optimal parameters for the ROC method
    ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                    privileged_groups=privileged_groups, 
                                    low_class_thresh=0.01, high_class_thresh=0.99,
                                    num_class_thresh=100, num_ROC_margin=50,
                                    metric_name=metric_name,
                                    metric_ub=metric_ub, metric_lb=metric_lb)
    ROC = ROC.fit(dataset_orig_train, dataset_orig_train_pred)
    # print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
    # print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

    # Metrics for the train set
    fav_inds = dataset_orig_train_pred.scores > best_class_thresh
    dataset_orig_train_pred.labels[fav_inds] = dataset_orig_train_pred.favorable_label
    dataset_orig_train_pred.labels[~fav_inds] = dataset_orig_train_pred.unfavorable_label

    # display(Markdown("#### train set"))
    # display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

    metric_train_bef = compute_metrics(dataset_orig_train, dataset_orig_train_pred, 
                    unprivileged_groups, privileged_groups)

    # Transform the validation set
    dataset_transf_train_pred = ROC.predict(dataset_orig_train_pred)

    # display(Markdown("#### train set"))
    # display(Markdown("##### Transformed predictions - With fairness constraints"))
    metric_train_aft = compute_metrics(dataset_orig_train, dataset_transf_train_pred, 
                    unprivileged_groups, privileged_groups)
    # Testing: Check if the metric optimized has not become worse
    assert np.abs(metric_train_aft[metric_name]) <= np.abs(metric_train_bef[metric_name])

    # Metrics for the test set
    fav_inds = dataset_orig_test_pred.scores > best_class_thresh
    dataset_orig_test_pred.labels[fav_inds] = dataset_orig_test_pred.favorable_label
    dataset_orig_test_pred.labels[~fav_inds] = dataset_orig_test_pred.unfavorable_label

    # display(Markdown("#### Test set"))
    # display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

    metric_test_bef = compute_metrics(dataset_orig_test, dataset_orig_test_pred, 
                    unprivileged_groups, privileged_groups)
    # Metrics for the transformed test set
    dataset_transf_test_pred = ROC.predict(dataset_orig_test_pred)

    # display(Markdown("#### Test set"))
    # display(Markdown("##### Transformed predictions - With fairness constraints"))
    metric_test_aft = compute_metrics(dataset_orig_test, dataset_transf_test_pred, 
                    unprivileged_groups, privileged_groups)

    #自己计算error, 不是balanced accuracy！！！
    print("K = {}, budget = {}".format(K, budget))
    print("The Error for the test dataset is {:.4}".format(np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels)))
    print("The Equal opportunity difference for the test dataset is {:.4}".format(metric_test_aft["Equal opportunity difference"]))
    experiments_info["K = {}, budget = {}".format(K, budget)] = {"Error": np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels), "Equal opportunity difference": metric_test_aft["Equal opportunity difference"]}

Balanced accuracy = 0.7811
Statistical parity difference = -0.2094
Disparate impact = 0.6231
Average odds difference = -0.1167
Equal opportunity difference = -0.0667
Theil index = 0.2996
Balanced accuracy = 0.7643
Statistical parity difference = -0.1467
Disparate impact = 0.7859
Average odds difference = -0.0686
Equal opportunity difference = 0.0143
Theil index = 0.1904
Balanced accuracy = 0.6536
Statistical parity difference = -0.1963
Disparate impact = 0.6204
Average odds difference = -0.1514
Equal opportunity difference = -0.2271
Theil index = 0.4305
Balanced accuracy = 0.6710
Statistical parity difference = 0.0293
Disparate impact = 1.0478
Average odds difference = 0.1120
Equal opportunity difference = -0.0491
Theil index = 0.2723
K = 1, budget = 0.1
The Error for the test dataset is 0.31
The Equal opportunity difference for the test dataset is -0.04909
Balanced accuracy = 0.6465
Statistical parity difference = 0.0258
Disparate impact = 1.0435
Average odds difference = -0.0701
Equa

In [10]:
experiments_info

{'K = 1, budget = 0.1': {'Error': 0.31,
  'Equal opportunity difference': -0.04909496652615919},
 'K = 2, budget = 0.1': {'Error': 0.305,
  'Equal opportunity difference': 0.27573027573027575},
 'K = 3, budget = 0.1': {'Error': 0.31,
  'Equal opportunity difference': 0.10576923076923073},
 'K = 4, budget = 0.1': {'Error': 0.295,
  'Equal opportunity difference': 0.06661206661206664},
 'K = 5, budget = 0.1': {'Error': 0.355,
  'Equal opportunity difference': -0.10128458498023718}}